## 💼 Projeto – K-Means

Uma empresa de concessão de crédito para pequenas empresas possui um catálogo de seus clientes (PJ)  
**com informações como idade da empresa, faturamento mensal, nível de inovação, entre outras**.  

Para que esta empresa possa oferecer um atendimento mais apropriado para cada tipo de cliente,  
eles **desejam agrupar estes clientes conforme estas características**.

---

Desta forma, para que seja possível classificar novos clientes,  
iremos construir um **algoritmo de clusterização** que agrupe os clientes em **segmentos**,  
com base nas **informações disponíveis** sobre o mesmo.


In [2]:
# EDA e visualizacao de dados
import pandas as pd
import plotly.express as px
pd.set_option('display.float_format', lambda x: '%.2f' % x)

# ML
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, pairwise_distances
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer

# otimizacao HP
import optuna

import os
os.environ["OMP_NUM_THREADS"] = "2"

### Carregar os Dados

In [3]:
# carga de dados
df_clientes = pd.read_csv('./datasets/dataset_segmento_clientes.csv')

In [4]:
# visualizar a estrutura
df_clientes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   atividade_economica     500 non-null    object 
 1   faturamento_mensal      500 non-null    float64
 2   numero_de_funcionarios  500 non-null    int64  
 3   localizacao             500 non-null    object 
 4   idade                   500 non-null    int64  
 5   inovacao                500 non-null    int64  
 6   segmento_de_cliente     500 non-null    object 
dtypes: float64(1), int64(3), object(3)
memory usage: 27.5+ KB


In [5]:
# visualizar os primeiros registros
df_clientes.head(10)

,atividade_economica,faturamento_mensal,numero_de_funcionarios,localizacao,idade,inovacao,segmento_de_cliente
0,Comércio,713109.95,12,Rio de Janeiro,6,1,Bronze
1,Comércio,790714.38,9,São Paulo,15,0,Bronze
2,Comércio,1197239.33,17,São Paulo,4,9,Silver
3,Indústria,449185.78,15,São Paulo,6,0,Starter
4,Agronegócio,1006373.16,15,São Paulo,15,8,Silver
5,Serviços,1629562.41,16,Rio de Janeiro,11,4,Silver
6,Serviços,771179.95,13,Vitória,0,1,Starter
7,Serviços,707837.61,16,São Paulo,10,6,Silver
8,Comércio,888983.66,17,Belo Horizonte,10,1,Bronze
9,Indústria,1098512.64,13,Rio de Janeiro,9,3,Bronze


### EDA

In [6]:
# distribuicao da variavel inovacao
percentual_inovacao = df_clientes.value_counts('inovacao') / len(df_clientes) * 100
px.bar(percentual_inovacao, color=percentual_inovacao.index)

Teste ANOVA (Análise de Variância)

Objetivo:  
Verificar se há variações significativas na média de faturamento mensal  
para diferentes níveis de inovação.

Suposições / Pressupostos:  
- Observações independentes  
- Variável dependente é contínua  
- Segue uma distribuição normal  
- Homogeneidade das variâncias  
- Amostras sejam de tamanhos iguais

Checar se as variâncias (faturamento) entre os grupos (inovação) são homogêneas

Aplicar Teste de Bartlett

H₀ – Variâncias são iguais  
H₁ – Variâncias não são iguais

In [7]:
from scipy.stats import bartlett

# separar os dados de faturamento em grupos com base na coluna inovacao
dados_agrupados = [df_clientes['faturamento_mensal'][df_clientes['inovacao'] == grupo] for grupo in df_clientes['inovacao'].unique()]

# executar o teste de bartlett
bartlett_test_statistic, bartlett_p_value = bartlett(*dados_agrupados)

# exibindo os resultados
print(f"Estatística do Teste de Bartlett: {bartlett_test_statistic}")
print(f"P-Value do Teste de Bartlett: {bartlett_p_value}")

Estatística do Teste de Bartlett: 10.901203117231173
P-Value do Teste de Bartlett: 0.28254182954905804


Executar o Teste de Shapiro-Wilk

Verificar se os dados seguem uma distribuição normal

H₀ – Segue uma distribuição normal  
H₁ – Não segue uma distribuição normal

In [8]:
from scipy.stats import shapiro

# executar o teste
shapiro_test_statistic, shapiro_p_value = shapiro(df_clientes['faturamento_mensal'])

# exibindo os resultados
print(f"Estatística do Teste de Shapiro-Wilk: {shapiro_test_statistic}")
print(f"P-Value do Teste de Shapiro-Wilk: {shapiro_p_value}")

Estatística do Teste de Shapiro-Wilk: 0.9959857602472711
P-Value do Teste de Shapiro-Wilk: 0.23513451034389005


Aplicar a ANOVA de Welch, pois as amostras são de tamanhos diferentes

H₀ – Não há diferenças significativas entre as médias dos grupos  
H₁ – Há pelo menos uma diferença significativa entre as médias dos grupos

In [9]:
from pingouin import welch_anova

aov = welch_anova(dv='faturamento_mensal', between='inovacao', data=df_clientes)

# exibindo os resultados
print(f'Estatística do Teste de ANOVA Welch: {aov.loc[0, "F"]}')
print(f'P-Value do Teste de ANOVA Welch: {aov.loc[0, "p-unc"]}')

Estatística do Teste de ANOVA Welch: 1.1269836194061693
P-Value do Teste de ANOVA Welch: 0.34526211273912577


### Treinar o algoritmo K-Means

In [10]:
# selecionar as colunas para clusterizacao
X = df_clientes.copy()

# separando variaveis numericas, categoricas e ordinais
numeric_features = ['faturamento_mensal', 'numero_de_funcionarios', 'idade']
categorical_features = ['localizacao', 'atividade_economica']
ordinal_features = ['inovacao']

# aplicar transformacoes por tipo
numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder()
ordinal_transformer = OrdinalEncoder()

# criar o preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features),
        ('ord', ordinal_transformer, ordinal_features)
    ]
)

# transformar os dados
X_transformed = preprocessor.fit_transform(X)

In [11]:
X_transformed

array([[-0.74634498, -0.54179191, -1.10058849, ...,  0.        ,
         0.        ,  1.        ],
       [-0.56165548, -1.5035527 ,  1.94344851, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.40582654,  1.06114274, -1.77704115, ...,  0.        ,
         0.        ,  9.        ],
       ...,
       [ 2.8196246 , -1.18296577,  0.25231684, ...,  0.        ,
         1.        ,  0.        ],
       [ 1.03321411, -0.54179191, -1.43881482, ...,  0.        ,
         0.        ,  3.        ],
       [-2.03011486, -0.22120498, -1.77704115, ...,  1.        ,
         0.        ,  9.        ]], shape=(500, 12))

In [12]:
# optuna para otimizacao de hiperparametros
def kmeans_objective(trial):
    # definir os hiperparametros a serem ajustados
    n_clusters = trial.suggest_int('n_clusters', 3, 10)
    distance_metric = trial.suggest_categorical('distance_metric', ['euclidean', 'minkowski'])

    # criar o modelo
    modelo_kmeans = KMeans(n_clusters=n_clusters, random_state=51)

    # treinar o modelo
    modelo_kmeans.fit(X_transformed)

    # calcular o Shilouette Score
    distances = pairwise_distances(X_transformed, metric=distance_metric)
    silhouette_avg = silhouette_score(distances, modelo_kmeans.labels_)

    return silhouette_avg

In [13]:
# criar um estudo do Optuna
search_space = {'n_clusters': [3, 4, 5, 6, 7, 8, 9, 10], 'distance_metric': ['euclidean', 'minkowski']}
sampler = optuna.samplers.GridSampler(search_space=search_space)
estudo_kmeans = optuna.create_study(direction='maximize', sampler=sampler)

# rodar estudo
estudo_kmeans.optimize(kmeans_objective, n_trials=100)

[I 2025-04-09 16:41:53,742] A new study created in memory with name: no-name-9fc29797-fb1a-4ac3-9a7b-5b234c53e02c
c:\Users\guilh\miniconda3\envs\Projeto_ML\Lib\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning:

KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.

[I 2025-04-09 16:41:53,845] Trial 0 finished with value: 0.3847879496502924 and parameters: {'n_clusters': 4, 'distance_metric': 'euclidean'}. Best is trial 0 with value: 0.3847879496502924.
c:\Users\guilh\miniconda3\envs\Projeto_ML\Lib\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning:

KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.

[I 2025-04-09 16:41:53,871] Trial 1 finished with value: 0.11230048381589089 and parameters: {'n_clusters': 9, 'dis

In [14]:
# melhor configuracao encontrada pelo Optuna
best_params = estudo_kmeans.best_params

# instanciando o modelo kmeans com os melhores parametros
best_kmeans = KMeans(n_clusters=best_params['n_clusters'], random_state=51)
best_kmeans.fit(X_transformed)

# calcular o Shilouette Score
distances = pairwise_distances(X_transformed, metric=best_params['distance_metric'])
best_silhouette = silhouette_score(distances, best_kmeans.labels_)

print(f'k (Numero de Clusters): {best_params["n_clusters"]}')
print(f'Metrica de Distancia Selecionada: {best_params["distance_metric"]}')
print(f'Silhouette Score: {best_silhouette}')


k (Numero de Clusters): 3
Metrica de Distancia Selecionada: minkowski
Silhouette Score: 0.4445458290999088


c:\Users\guilh\miniconda3\envs\Projeto_ML\Lib\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning:

KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.



In [15]:
# criar coluna com cluster escolhido
df_clientes['cluster'] = best_kmeans.labels_

In [17]:
# visualizar os primeiros registros
df_clientes.head(10)

,atividade_economica,faturamento_mensal,numero_de_funcionarios,localizacao,idade,inovacao,segmento_de_cliente,cluster
0,Comércio,713109.95,12,Rio de Janeiro,6,1,Bronze,0
1,Comércio,790714.38,9,São Paulo,15,0,Bronze,0
2,Comércio,1197239.33,17,São Paulo,4,9,Silver,1
3,Indústria,449185.78,15,São Paulo,6,0,Starter,0
4,Agronegócio,1006373.16,15,São Paulo,15,8,Silver,1
5,Serviços,1629562.41,16,Rio de Janeiro,11,4,Silver,2
6,Serviços,771179.95,13,Vitória,0,1,Starter,0
7,Serviços,707837.61,16,São Paulo,10,6,Silver,1
8,Comércio,888983.66,17,Belo Horizonte,10,1,Bronze,0
9,Indústria,1098512.64,13,Rio de Janeiro,9,3,Bronze,2


### Visualizar Resultados

In [18]:
# cruzar idade e faturamento, apresentando os clusters
px.scatter(df_clientes, x='idade', y='faturamento_mensal', color='cluster', title='Clusters de Clientes - Idade vs Faturamento Mensal')

In [19]:
# cruzar inovacao e faturamento, apresentando os clusters
px.scatter(df_clientes, x='inovacao', y='faturamento_mensal', color='cluster', title='Clusters de Clientes - Inovacao vs Faturamento Mensal')

In [20]:
# cruzar numero funcionarios e faturamento, apresentando os clusters
px.scatter(df_clientes, x='numero_de_funcionarios', y='faturamento_mensal', color='cluster', title='Clusters de Clientes - Funcionarios vs Faturamento Mensal')

### Salvar o Modelo e o Pipeline de Transformação

In [21]:
import joblib

# salvar o modelo
joblib.dump(best_kmeans, 'modelo_clusterizacao_clientes.pkl')

# salvar o pipeline
joblib.dump(preprocessor, 'pipeline_clusterizacao_clientes.pkl')

['pipeline_clusterizacao_clientes.pkl']

### Aplicação Batch no Gradio

In [22]:
import gradio as gr

modelo = joblib.load('./modelo_clusterizacao_clientes.pkl')
preprocessor = joblib.load('./pipeline_clusterizacao_clientes.pkl')

def clustering(arquivo):
    # carregar csv em um dataframe
    df_empresas = pd.read_csv(arquivo.name)

    # transformar os dados do df para o formato que o kmeans precisa
    X_transformed = preprocessor.fit_transform(df_empresas)

    # treinar o modelo
    modelo.fit(X_transformed)

    # criar a coluna cluster no df
    df_empresas['cluster'] = modelo.labels_
    df_empresas.to_csv('./clusters.csv', index=False)

    return './clusters.csv'

In [23]:
# criar a interface
app = gr.Interface(
    clustering,
    gr.File(file_types=['.csv']),
    'file'
)

# rodar a aplicacao
app.launch()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


c:\Users\guilh\miniconda3\envs\Projeto_ML\Lib\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning:

KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.

